# 第3章 推荐系统冷启动问题

## 冷启动问题简介

冷启动问题主要分3类：  
1. 用户冷启动：主要解决如何给新用户做个性化推荐的问题
2. 物品冷启动：主要解决如何将新的物品推荐给可能对它感兴趣的用户
3. 系统冷启动：主要解决如何在一个新开发的网站上（还没有用户，也没有用户行为，只有一些物品的信息）设计个性化推荐，从而在网站刚发布时就让用户体验到个性化推荐服务

针对2个不同的冷启动问题，有如下解决方案：  
1. 提供非个性化的推荐：最简单的例子是热门排行榜，等到用户数据收集到一定的时候，再切换为个性化推荐
2. 利用用户注册时提供的年龄、性别等数据做粗粒度的个性化
3. 利用用户的社交网络帐号登录（需要用户授权），导入用户在社交网站上的好友信息，然后给用户推荐其好友喜欢的物品（UserCF算法）
4. 要求用户在登录时对一些物品进行反馈，收集用户对这些物品的兴趣信息，然后给用户推荐那些和这些物品相似的物品（ItemCF算法和LFM算法）
5. 对于新加入的物品，可以利用内容信息，将它们推荐给喜欢过和它们相似的物品的用户（ItemCF算法）
6. 在系统冷启动时，引入专家的知识，通过一定的高效方式迅速建立起物品的相关度表（基于图的推荐算法，PersonalRank算法）

## 利用用户注册信息

用户的注册信息分3种：
1. 人口统计学信息：包括用户的年龄、性别、职业、民族、学历和居住地
2. 用户兴趣的描述：让用户用文字描述他们的兴趣
3. 从其他网站导入的用户站外行为数据：比如用户通过微博的帐号登录，就可以在得到用户同意的情况下获取用户在第三方平台的一些行为数据和社交网络数据。

基于注册信息的个性化推荐流程基本如下：  
1. 获取用户的注册信息
2. 根据用户的注册信息对用户分类
3. 给用户推荐他所属分类中用户喜欢的物品

&emsp;&emsp;基于用户注册信息的推荐算法其**核心问题**是计算每种特征的用户喜欢的物品。对于每种特征$f$，计算具有这种特征的用户对各个物品的喜好程度$p(f, i)$。$p(f, i)$定义为物品$i$在具有$f$的特征的用户中的热门程度：$$p(f, i)=\left|N(i) \cap U(f)\right|$$其中$N(i)$是喜欢该物品的用户集合，$U(f)$是具有特征$f$的用户集合。  
&emsp;&emsp;为解决用户发现他们不容易发现的物品，将$p(f,i)$定义为喜欢物品$i$的用户中具有特征$f$的比例：$$p(f, i)=\frac{|N(i) \cap U(f)|}{|N(i)| + \alpha}$$其中参数$\alpha$的目的是解决数据稀疏的问题。

## 选择合适的物品启动用户的兴趣

&emsp;&emsp;在新用户第一次访问推荐系统时，不立即给用户展示推荐系统，而是给用户提供一些物品，让用户反馈他们对这些物品的兴趣，然后根据用户反馈，提供个性化推荐。  

能够用来启动用户兴趣的物品具有以下特点：  
1. 比较热门：让用户对热门的物品进行反馈
2. 具有代表性和区分性：启动用户兴趣的物品不能是大众化或老少咸宜的，因为这样的物品对用户的兴趣没有区分新。
3. 启动物品集合需要有多样性：在冷启动时，为了匹配多样的兴趣，需要提供具有很高覆盖率的启动物品集合，这些物品能覆盖几乎所有主流的用户兴趣。

&emsp;&emsp;首先，给定一群用户，用这群用户对物品评分的方差度量这群用户兴趣的一致程度。如果方差很小，说明这一群用户的兴趣不太一致，也就是说物品具有比较大的区分度，反之则说明，这群用户的兴趣比较一致。  
&emsp;&emsp;令$\sigma_{u \in U'}$为用户集合$U'$中所有评分的方差，通过如下方式度量一个物品的区分度$D(i)$：$$D(i)=\sigma_{u \in N^+(i)} + \sigma_{u \in N^-(i)} + \sigma_{u \in \bar{N}(i)}$$